In [2]:
!pip install tensorflow pandas scipy -q
!pip install -U "seaborn" --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 620.7/620.7 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.5/24.5 MB 85.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 125.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 124.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 18.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 6.3 MB/s eta 0:00:00


In [4]:
# !pip install tensorflow pandas scipy -q
# !pip install -U "seaborn" --quiet

import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.io import savemat

# --- Main Configuration ---
# CHANGE THIS VARIABLE to 'exp_power' or 'sinh_cosh_power' before running
MODEL_TYPE = 'exp_power' # or 'sinh_cosh_power'

# --- Setup, Model Definitions, Helper/Physics Functions ---
tf.keras.backend.set_floatx('float64')
DEVICE = "/GPU:0" if tf.config.list_physical_devices('GPU') else "/CPU:0"
print(f"Using device: {DEVICE} for model type: {MODEL_TYPE}")

ARG_CLIP_MIN = tf.constant(-10.0, dtype=tf.float64)
ARG_CLIP_MAX = tf.constant(10.0, dtype=tf.float64)

@tf.function
def smooth_relu(x, beta=20.0):
    return tf.nn.softplus(beta * x) / beta

# --- Model for Exponential Power Terms Only ---
class StrainEnergy_ExpPower_C(tf.keras.Model):
    def __init__(self):
        super().__init__(name="StrainEnergy_ExpPower_C")
        self.raw_log_k1=self.add_weight(name="raw_log_k1",shape=(),dtype=tf.float64,initializer=tf.keras.initializers.RandomNormal(mean=0.0,stddev=0.01),trainable=True)
        self.raw_log_k2=self.add_weight(name="raw_log_k2",shape=(),dtype=tf.float64,initializer=tf.keras.initializers.RandomNormal(mean=0.0,stddev=0.01),trainable=True)
        self.raw_log_i1=self.add_weight(name="raw_log_i1",shape=(),dtype=tf.float64,initializer=tf.keras.initializers.RandomNormal(mean=0.0,stddev=0.01),trainable=True)
        self.raw_log_i2=self.add_weight(name="raw_log_i2",shape=(),dtype=tf.float64,initializer=tf.keras.initializers.RandomNormal(mean=0.0,stddev=0.01),trainable=True)
        self.raw_log_a1=self.add_weight(name="raw_log_a1",shape=(),dtype=tf.float64,initializer=tf.keras.initializers.Constant(tf.math.log(0.1)),trainable=True)
        self.raw_log_a2=self.add_weight(name="raw_log_a2",shape=(),dtype=tf.float64,initializer=tf.keras.initializers.Constant(tf.math.log(0.1)),trainable=True)
        self.raw_log_k3=self.add_weight(name="raw_log_k3",shape=(),dtype=tf.float64,initializer=tf.keras.initializers.RandomNormal(mean=0.0,stddev=0.01),trainable=True)
        self.raw_log_k4=self.add_weight(name="raw_log_k4",shape=(),dtype=tf.float64,initializer=tf.keras.initializers.RandomNormal(mean=0.0,stddev=0.01),trainable=True)
        self.raw_log_i3=self.add_weight(name="raw_log_i3",shape=(),dtype=tf.float64,initializer=tf.keras.initializers.RandomNormal(mean=0.0,stddev=0.01),trainable=True)
        self.raw_log_i4=self.add_weight(name="raw_log_i4",shape=(),dtype=tf.float64,initializer=tf.keras.initializers.RandomNormal(mean=0.0,stddev=0.01),trainable=True)
        self.raw_log_a3=self.add_weight(name="raw_log_a3",shape=(),dtype=tf.float64,initializer=tf.keras.initializers.Constant(tf.math.log(0.1)),trainable=True)
        self.raw_log_a4=self.add_weight(name="raw_log_a4",shape=(),dtype=tf.float64,initializer=tf.keras.initializers.Constant(tf.math.log(0.1)),trainable=True)
        self.raw_log_a3_prime=self.add_weight(name="raw_log_a3_prime",shape=(),dtype=tf.float64,initializer=tf.keras.initializers.Constant(tf.math.log(0.2)),trainable=True)
        self.raw_log_a4_prime=self.add_weight(name="raw_log_a4_prime",shape=(),dtype=tf.float64,initializer=tf.keras.initializers.Constant(tf.math.log(0.2)),trainable=True)
        self.raw_log_k9=self.add_weight(name="raw_log_k9",shape=(),dtype=tf.float64,initializer=tf.keras.initializers.RandomNormal(mean=0.0,stddev=0.01),trainable=True)
        self.raw_log_k10=self.add_weight(name="raw_log_k10",shape=(),dtype=tf.float64,initializer=tf.keras.initializers.RandomNormal(mean=0.0,stddev=0.01),trainable=True)
        self.raw_log_b1=self.add_weight(name="raw_log_b1",shape=(),dtype=tf.float64,initializer=tf.keras.initializers.Constant(tf.math.log(0.01)),trainable=True)
        self.raw_log_b2=self.add_weight(name="raw_log_b2",shape=(),dtype=tf.float64,initializer=tf.keras.initializers.Constant(tf.math.log(0.01)),trainable=True)
        self.raw_log_k11=self.add_weight(name="raw_log_k11",shape=(),dtype=tf.float64,initializer=tf.keras.initializers.RandomNormal(mean=0.0,stddev=0.01),trainable=True)
        self.raw_log_k12=self.add_weight(name="raw_log_k12",shape=(),dtype=tf.float64,initializer=tf.keras.initializers.RandomNormal(mean=0.0,stddev=0.01),trainable=True)
        self.raw_log_b3=self.add_weight(name="raw_log_b3",shape=(),dtype=tf.float64,initializer=tf.keras.initializers.Constant(tf.math.log(0.1)),trainable=True)
        self.raw_log_b4=self.add_weight(name="raw_log_b4",shape=(),dtype=tf.float64,initializer=tf.keras.initializers.Constant(tf.math.log(0.1)),trainable=True)
        self.raw_log_b3_prime=self.add_weight(name="raw_log_b3_prime",shape=(),dtype=tf.float64,initializer=tf.keras.initializers.Constant(tf.math.log(0.2)),trainable=True)
        self.raw_log_b4_prime=self.add_weight(name="raw_log_b4_prime",shape=(),dtype=tf.float64,initializer=tf.keras.initializers.Constant(tf.math.log(0.2)),trainable=True)
        self.three=tf.constant(3.0,dtype=tf.float64); self.one=tf.constant(1.0,dtype=tf.float64); self.pow_base_epsilon=tf.constant(1e-8,dtype=tf.float64)
    def _term_power_law(self, I, k, i, c, ref_val): return c * tf.pow(smooth_relu(tf.pow(I, k) - tf.pow(ref_val, k)) + self.pow_base_epsilon, i)
    def _term_exponential(self, I, k, i, ic, oc, ref_val): return oc * (tf.exp(tf.clip_by_value(ic * tf.pow(smooth_relu(tf.pow(I, k) - tf.pow(ref_val, k)) + self.pow_base_epsilon, i), ARG_CLIP_MIN, ARG_CLIP_MAX)) - 1.0)
    def _term_identity_scaled(self, I, k, c, ref_val): return c * (tf.pow(I, k) - tf.pow(ref_val, k))
    def _term_exponential_no_i(self, I, k, ic, oc, ref_val): return oc * (tf.exp(tf.clip_by_value(ic * (tf.pow(I, k) - tf.pow(ref_val, k)), ARG_CLIP_MIN, ARG_CLIP_MAX)) - 1.0)
    def call(self, I1, I2, I4, I6):
        k1=1.0+tf.exp(self.raw_log_k1); k2=1.5+tf.exp(self.raw_log_k2); k3=1.0+tf.exp(self.raw_log_k3); k4=1.5+tf.exp(self.raw_log_k4)
        i1=1.0+tf.exp(self.raw_log_i1); i2=1.0+tf.exp(self.raw_log_i2); i3=1.0+tf.exp(self.raw_log_i3); i4=1.0+tf.exp(self.raw_log_i4)
        a1=tf.exp(self.raw_log_a1); a2=tf.exp(self.raw_log_a2); a3=tf.exp(self.raw_log_a3); a4=tf.exp(self.raw_log_a4)
        a3_prime=tf.exp(self.raw_log_a3_prime); a4_prime=tf.exp(self.raw_log_a4_prime)
        k9=1.0+tf.exp(self.raw_log_k9); k10=1.5+tf.exp(self.raw_log_k10); k11=1.0+tf.exp(self.raw_log_k11); k12=1.5+tf.exp(self.raw_log_k12)
        b1=tf.exp(self.raw_log_b1); b2=tf.exp(self.raw_log_b2); b3=tf.exp(self.raw_log_b3); b4=tf.exp(self.raw_log_b4)
        b3_prime=tf.exp(self.raw_log_b3_prime); b4_prime=tf.exp(self.raw_log_b4_prime)
        W = tf.zeros_like(I1, dtype=tf.float64)
        W += self._term_power_law(I1, k1, i1, a1, self.three); W += self._term_power_law(I2, k2, i2, a2, self.three)
        W += self._term_exponential(I1, k3, i3, a3_prime, a3, self.three); W += self._term_exponential(I2, k4, i4, a4_prime, a4, self.three)
        W += self._term_identity_scaled(I4, k9, b1, self.one); W += self._term_identity_scaled(I6, k10, b2, self.one)
        W += self._term_exponential_no_i(I4, k11, b3_prime, b3, self.one); W += self._term_exponential_no_i(I6, k12, b4_prime, b4, self.one)
        return W

# --- Model for Sinh-Cosh Power Terms Only ---
class StrainEnergy_SinhCoshPower_C(tf.keras.Model):
    def __init__(self):
        super().__init__(name="StrainEnergy_SinhCoshPower_C")
        self.raw_log_k1=self.add_weight(name="raw_log_k1",shape=(),dtype=tf.float64,initializer=tf.keras.initializers.RandomNormal(mean=0.0,stddev=0.01),trainable=True)
        self.raw_log_k2=self.add_weight(name="raw_log_k2",shape=(),dtype=tf.float64,initializer=tf.keras.initializers.RandomNormal(mean=0.0,stddev=0.01),trainable=True)
        self.raw_log_i1=self.add_weight(name="raw_log_i1",shape=(),dtype=tf.float64,initializer=tf.keras.initializers.RandomNormal(mean=0.0,stddev=0.01),trainable=True)
        self.raw_log_i2=self.add_weight(name="raw_log_i2",shape=(),dtype=tf.float64,initializer=tf.keras.initializers.RandomNormal(mean=0.0,stddev=0.01),trainable=True)
        self.raw_log_a1=self.add_weight(name="raw_log_a1",shape=(),dtype=tf.float64,initializer=tf.keras.initializers.Constant(tf.math.log(0.1)),trainable=True)
        self.raw_log_a2=self.add_weight(name="raw_log_a2",shape=(),dtype=tf.float64,initializer=tf.keras.initializers.Constant(tf.math.log(0.1)),trainable=True)
        self.raw_log_k5=self.add_weight(name="raw_log_k5",shape=(),dtype=tf.float64,initializer=tf.keras.initializers.RandomNormal(mean=0.0,stddev=0.01),trainable=True)
        self.raw_log_k6=self.add_weight(name="raw_log_k6",shape=(),dtype=tf.float64,initializer=tf.keras.initializers.RandomNormal(mean=0.0,stddev=0.01),trainable=True)
        self.raw_log_i5=self.add_weight(name="raw_log_i5",shape=(),dtype=tf.float64,initializer=tf.keras.initializers.RandomNormal(mean=0.0,stddev=0.01),trainable=True)
        self.raw_log_i6=self.add_weight(name="raw_log_i6",shape=(),dtype=tf.float64,initializer=tf.keras.initializers.RandomNormal(mean=0.0,stddev=0.01),trainable=True)
        self.raw_log_a5=self.add_weight(name="raw_log_a5",shape=(),dtype=tf.float64,initializer=tf.keras.initializers.Constant(tf.math.log(0.1)),trainable=True)
        self.raw_log_a6=self.add_weight(name="raw_log_a6",shape=(),dtype=tf.float64,initializer=tf.keras.initializers.Constant(tf.math.log(0.1)),trainable=True)
        self.raw_log_a5_prime=self.add_weight(name="raw_log_a5_prime",shape=(),dtype=tf.float64,initializer=tf.keras.initializers.Constant(tf.math.log(0.2)),trainable=True)
        self.raw_log_a6_prime=self.add_weight(name="raw_log_a6_prime",shape=(),dtype=tf.float64,initializer=tf.keras.initializers.Constant(tf.math.log(0.2)),trainable=True)
        self.raw_log_k7=self.add_weight(name="raw_log_k7",shape=(),dtype=tf.float64,initializer=tf.keras.initializers.RandomNormal(mean=0.0,stddev=0.01),trainable=True)
        self.raw_log_k8=self.add_weight(name="raw_log_k8",shape=(),dtype=tf.float64,initializer=tf.keras.initializers.RandomNormal(mean=0.0,stddev=0.01),trainable=True)
        self.raw_log_i7=self.add_weight(name="raw_log_i7",shape=(),dtype=tf.float64,initializer=tf.keras.initializers.RandomNormal(mean=0.0,stddev=0.01),trainable=True)
        self.raw_log_i8=self.add_weight(name="raw_log_i8",shape=(),dtype=tf.float64,initializer=tf.keras.initializers.RandomNormal(mean=0.0,stddev=0.01),trainable=True)
        self.raw_log_a7=self.add_weight(name="raw_log_a7",shape=(),dtype=tf.float64,initializer=tf.keras.initializers.Constant(tf.math.log(0.1)),trainable=True)
        self.raw_log_a8=self.add_weight(name="raw_log_a8",shape=(),dtype=tf.float64,initializer=tf.keras.initializers.Constant(tf.math.log(0.1)),trainable=True)
        self.raw_log_a7_prime=self.add_weight(name="raw_log_a7_prime",shape=(),dtype=tf.float64,initializer=tf.keras.initializers.Constant(tf.math.log(0.2)),trainable=True)
        self.raw_log_a8_prime=self.add_weight(name="raw_log_a8_prime",shape=(),dtype=tf.float64,initializer=tf.keras.initializers.Constant(tf.math.log(0.2)),trainable=True)
        self.raw_log_k13=self.add_weight(name="raw_log_k13",shape=(),dtype=tf.float64,initializer=tf.keras.initializers.RandomNormal(mean=0.0,stddev=0.01),trainable=True)
        self.raw_log_k14=self.add_weight(name="raw_log_k14",shape=(),dtype=tf.float64,initializer=tf.keras.initializers.RandomNormal(mean=0.0,stddev=0.01),trainable=True)
        self.raw_log_b5=self.add_weight(name="raw_log_b5",shape=(),dtype=tf.float64,initializer=tf.keras.initializers.Constant(tf.math.log(0.1)),trainable=True)
        self.raw_log_b6=self.add_weight(name="raw_log_b6",shape=(),dtype=tf.float64,initializer=tf.keras.initializers.Constant(tf.math.log(0.1)),trainable=True)
        self.raw_log_b5_prime=self.add_weight(name="raw_log_b5_prime",shape=(),dtype=tf.float64,initializer=tf.keras.initializers.Constant(tf.math.log(0.2)),trainable=True)
        self.raw_log_b6_prime=self.add_weight(name="raw_log_b6_prime",shape=(),dtype=tf.float64,initializer=tf.keras.initializers.Constant(tf.math.log(0.2)),trainable=True)
        self.raw_log_k15=self.add_weight(name="raw_log_k15",shape=(),dtype=tf.float64,initializer=tf.keras.initializers.RandomNormal(mean=0.0,stddev=0.01),trainable=True)
        self.raw_log_k16=self.add_weight(name="raw_log_k16",shape=(),dtype=tf.float64,initializer=tf.keras.initializers.RandomNormal(mean=0.0,stddev=0.01),trainable=True)
        self.raw_log_b7=self.add_weight(name="raw_log_b7",shape=(),dtype=tf.float64,initializer=tf.keras.initializers.Constant(tf.math.log(0.1)),trainable=True)
        self.raw_log_b8=self.add_weight(name="raw_log_b8",shape=(),dtype=tf.float64,initializer=tf.keras.initializers.Constant(tf.math.log(0.1)),trainable=True)
        self.raw_log_b7_prime=self.add_weight(name="raw_log_b7_prime",shape=(),dtype=tf.float64,initializer=tf.keras.initializers.Constant(tf.math.log(0.2)),trainable=True)
        self.raw_log_b8_prime=self.add_weight(name="raw_log_b8_prime",shape=(),dtype=tf.float64,initializer=tf.keras.initializers.Constant(tf.math.log(0.2)),trainable=True)
        self.three=tf.constant(3.0,dtype=tf.float64); self.one=tf.constant(1.0,dtype=tf.float64); self.pow_base_epsilon=tf.constant(1e-8,dtype=tf.float64)
    def _term_power_law(self, I, k, i, c, ref_val): return c * tf.pow(smooth_relu(tf.pow(I, k) - tf.pow(ref_val, k)) + self.pow_base_epsilon, i)
    def _term_cosh_minus_one_with_i(self, I, k, i, ic, oc, ref_val): return oc * (tf.cosh(tf.clip_by_value(ic * tf.pow(smooth_relu(tf.pow(I, k) - tf.pow(ref_val, k)) + self.pow_base_epsilon, i), ARG_CLIP_MIN, ARG_CLIP_MAX)) - 1.0)
    def _term_sinh_with_i(self, I, k, i, ic, oc, ref_val): return oc * tf.sinh(tf.clip_by_value(ic * tf.pow(smooth_relu(tf.pow(I, k) - tf.pow(ref_val, k)) + self.pow_base_epsilon, i), ARG_CLIP_MIN, ARG_CLIP_MAX))
    def _term_cosh_minus_one(self, I, k, ic, oc, ref_val): return oc * (tf.cosh(tf.clip_by_value(ic * (tf.pow(I, k) - tf.pow(ref_val, k)), ARG_CLIP_MIN, ARG_CLIP_MAX)) - 1.0)
    def _term_sinh(self, I, k, ic, oc, ref_val): return oc * tf.sinh(tf.clip_by_value(ic * (tf.pow(I, k) - tf.pow(ref_val, k)), ARG_CLIP_MIN, ARG_CLIP_MAX))
    def call(self, I1, I2, I4, I6):
        k1=1.0+tf.exp(self.raw_log_k1);k2=1.5+tf.exp(self.raw_log_k2);k5=1.0+tf.exp(self.raw_log_k5);k6=1.5+tf.exp(self.raw_log_k6);k7=1.0+tf.exp(self.raw_log_k7);k8=1.5+tf.exp(self.raw_log_k8);k13=1.0+tf.exp(self.raw_log_k13);k14=1.5+tf.exp(self.raw_log_k14);k15=1.0+tf.exp(self.raw_log_k15);k16=1.5+tf.exp(self.raw_log_k16)
        i1=1.0+tf.exp(self.raw_log_i1);i2=1.0+tf.exp(self.raw_log_i2);i5=1.0+tf.exp(self.raw_log_i5);i6=1.0+tf.exp(self.raw_log_i6);i7=1.0+tf.exp(self.raw_log_i7);i8=1.0+tf.exp(self.raw_log_i8)
        a1=tf.exp(self.raw_log_a1);a2=tf.exp(self.raw_log_a2);a5=tf.exp(self.raw_log_a5);a6=tf.exp(self.raw_log_a6);a7=tf.exp(self.raw_log_a7);a8=tf.exp(self.raw_log_a8)
        a5_prime=tf.exp(self.raw_log_a5_prime);a6_prime=tf.exp(self.raw_log_a6_prime);a7_prime=tf.exp(self.raw_log_a7_prime);a8_prime=tf.exp(self.raw_log_a8_prime)
        b5=tf.exp(self.raw_log_b5);b6=tf.exp(self.raw_log_b6);b7=tf.exp(self.raw_log_b7);b8=tf.exp(self.raw_log_b8)
        b5_prime=tf.exp(self.raw_log_b5_prime);b6_prime=tf.exp(self.raw_log_b6_prime);b7_prime=tf.exp(self.raw_log_b7_prime);b8_prime=tf.exp(self.raw_log_b8_prime)
        W = tf.zeros_like(I1,dtype=tf.float64)
        W += self._term_power_law(I1,k1,i1,a1,self.three); W += self._term_power_law(I2,k2,i2,a2,self.three)
        W += self._term_cosh_minus_one_with_i(I1,k5,i5,a5_prime,a5,self.three); W += self._term_cosh_minus_one_with_i(I2,k6,i6,a6_prime,a6,self.three)
        W += self._term_sinh_with_i(I1,k7,i7,a7_prime,a7,self.three); W += self._term_sinh_with_i(I2,k8,i8,a8_prime,a8,self.three)
        W += self._term_cosh_minus_one(I4,k13,b5_prime,b5,self.one); W += self._term_cosh_minus_one(I6,k14,b6_prime,b6,self.one)
        W += self._term_sinh(I4,k15,b7_prime,b7,self.one); W += self._term_sinh(I6,k16,b8_prime,b8,self.one)
        return W

# --- Universal Physics, Training, and Contribution Calculation Functions ---
@tf.function
def get_invariants_tf(lambda1, lambda2, lambda3):
    min_lambda_val = tf.constant(1e-6, dtype=tf.float64)
    lambda1 = tf.maximum(lambda1, min_lambda_val); lambda2 = tf.maximum(lambda2, min_lambda_val); lambda3 = tf.maximum(lambda3, min_lambda_val)
    l1s = tf.pow(lambda1, 2.0); l2s = tf.pow(lambda2, 2.0); l3s = tf.pow(lambda3, 2.0)
    I1 = l1s + l2s + l3s; I2 = tf.pow(lambda1*lambda2, 2.0) + tf.pow(lambda2*lambda3, 2.0) + tf.pow(lambda3*lambda1, 2.0)
    I4 = l1s; I6 = 1/l1s
    return I1, I2, I4, I6
@tf.function
def _calculate_raw_uniaxial_p11(l1, model, W_func):
    with tf.GradientTape(persistent=True) as tape:
        l1_t, l2_t, l3_t = l1, tf.pow(l1, -0.5), tf.pow(l1, -0.5)
        tape.watch([l1_t, l2_t])
        I1, I2, I4, I6 = get_invariants_tf(l1_t, l2_t, l3_t)
        W = W_func(I1, I2, I4, I6)
    dWdl1 = tape.gradient(W, l1_t); dWdl2 = tape.gradient(W, l2_t)
    del tape
    p = l2_t * dWdl2
    return dWdl1 - p / l1_t
@tf.function
def predict_uniaxial_p11(lambda1, model, W_func):
    p11_raw = _calculate_raw_uniaxial_p11(lambda1, model, W_func)
    p11_offset = _calculate_raw_uniaxial_p11(tf.constant([1.0], dtype=tf.float64), model, W_func)
    return p11_raw - p11_offset
@tf.function
def get_raw_biaxial_stresses(l1, l2, model, W_func):
    with tf.GradientTape(persistent=True) as tape:
        l1_t, l2_t = l1, l2; l3_t = 1.0 / (l1_t * l2_t)
        tape.watch([l1_t, l2_t, l3_t])
        I1, I2, I4, I6 = get_invariants_tf(l1_t, l2_t, l3_t)
        W = W_func(I1, I2, I4, I6)
    dWdl1, dWdl2, dWdl3 = tape.gradient(W, l1_t), tape.gradient(W, l2_t), tape.gradient(W, l3_t)
    del tape
    p = l3_t * dWdl3
    return dWdl1 - p / l1_t, dWdl2 - p / l2_t
@tf.function
def get_corrected_biaxial_stresses(l1, l2, model, W_func):
    p11_raw, p22_raw = get_raw_biaxial_stresses(l1, l2, model, W_func)
    # --- FIX: Explicitly cast constants to float64 ---
    p11_offset, p22_offset = get_raw_biaxial_stresses(tf.constant(1.0, dtype=tf.float64), tf.constant(1.0, dtype=tf.float64), model, W_func)
    return p11_raw - p11_offset, p22_raw - p22_offset
@tf.function
def predict_biaxial_p22(lambda2, model, W_func):
    l1_min = tf.ones_like(lambda2) * 0.5; l1_max = tf.ones_like(lambda2) * 1.5
    for _ in tf.range(50): # Bisection method loop
        l1_mid = (l1_min + l1_max) / 2.0
        p11_mid, _ = get_corrected_biaxial_stresses(l1_mid, lambda2, model, W_func)
        p11_min, _ = get_corrected_biaxial_stresses(l1_min, lambda2, model, W_func)
        is_same_sign = tf.sign(p11_mid) == tf.sign(p11_min)
        l1_min = tf.where(is_same_sign, l1_mid, l1_min)
        l1_max = tf.where(is_same_sign, l1_max, l1_mid)
    final_l1 = tf.stop_gradient((l1_min + l1_max) / 2.0)
    _, P22_final = get_corrected_biaxial_stresses(final_l1, lambda2, model, W_func)
    is_undeformed = tf.abs(lambda2 - 1.0) < 1e-9
    return tf.where(is_undeformed, tf.constant(0.0, dtype=tf.float64), P22_final)

# --- Experimental Data ---
exp_data_raw_uniaxial_cnf = np.array([[1.0000,0],[1.0708,0.3840],[1.2017,0.8987],[1.3125,1.1814],[1.4000,1.4093],[1.5125,1.6456],[1.6017,1.8608],[1.7125,2.1055],[1.8008,2.3122],[1.8883,2.5570],[1.9767,2.7848],[2.0883,3.1519],[2.1992,3.5274],[2.2867,3.8354],[2.3975,4.2532],[2.4383,4.4304],[2.4858,4.5949]])
exp_data_raw_biaxial_cnf = np.array([[1.0000,0],[1.3208,1.0506],[1.4017,1.2068],[1.5092,1.3840],[1.5983,1.5401],[1.7017,1.6835],[1.7842,1.7848],[1.8967,1.9662],[1.9792,2.1181],[2.0858,2.2911],[2.1708,2.4599],[2.2783,2.6962],[2.3825,2.9409],[2.4225,3.0549],[2.4867,3.2236]])
uniaxial_l1, uniaxial_p11 = [tf.constant(c, dtype=tf.float64) for c in exp_data_raw_uniaxial_cnf.T]
biaxial_l2, biaxial_p22 = [tf.constant(c, dtype=tf.float64) for c in exp_data_raw_biaxial_cnf.T]

# --- Model Selection ---
if MODEL_TYPE == 'exp_power':
    model = StrainEnergy_ExpPower_C()
elif MODEL_TYPE == 'sinh_cosh_power':
    model = StrainEnergy_SinhCoshPower_C()
else:
    raise ValueError("MODEL_TYPE must be 'exp_power' or 'sinh_cosh_power'")

# --- Training Setup ---
initial_learning_rate = 0.003
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(initial_learning_rate, decay_steps=2000, decay_rate=0.9, staircase=True)
optimizer = tf.keras.optimizers.Adam(learning_rate=lr_schedule)
loss_weight_biaxial = tf.constant(10.0, dtype=tf.float64)

# --- ENHANCEMENT: Using your more robust hybrid loss function ---
@tf.function
def train_step(uniaxial_l1, uniaxial_p11, biaxial_l2, biaxial_p22, loss_weight):
    with tf.GradientTape() as tape:
        # Uniaxial Loss (Hybrid)
        p11_pred = predict_uniaxial_p11(uniaxial_l1, model, model.call)
        relative_error_1 = (p11_pred[1:] - uniaxial_p11[1:]) / (uniaxial_p11[1:] + 1e-9)
        loss1_rel = tf.reduce_mean(tf.square(relative_error_1))
        loss1_abs = tf.reduce_mean(tf.square(p11_pred - uniaxial_p11))
        loss1 = loss1_rel + 0.1 * loss1_abs
        # Biaxial Loss (Hybrid)
        p22_pred = predict_biaxial_p22(biaxial_l2, model, model.call)
        relative_error_2 = (p22_pred[1:] - biaxial_p22[1:]) / (biaxial_p22[1:] + 1e-9)
        loss2_rel = tf.reduce_mean(tf.square(relative_error_2))
        loss2_abs = tf.reduce_mean(tf.square(p22_pred - biaxial_p22))
        loss2 = loss2_rel + 0.1 * loss2_abs
        total_loss = loss1 + loss_weight * loss2
    grads = tape.gradient(total_loss, model.trainable_variables)
    grads = [tf.clip_by_value(g, -1.0, 1.0) if g is not None else g for g in grads]
    optimizer.apply_gradients(zip(grads, model.trainable_variables))
    return total_loss, loss1_rel, loss2_rel

# --- Training Loop ---
epochs = 30000
print(f"--- Training {MODEL_TYPE} model for Calender Tissue ---");
for epoch in range(epochs):
    total_loss, loss1, loss2 = train_step(uniaxial_l1, uniaxial_p11, biaxial_l2, biaxial_p22, loss_weight_biaxial)
    if epoch % 1000 == 0:
        current_lr = lr_schedule(optimizer.iterations)
        print(f"Epoch {epoch:5d}, Loss: {total_loss:.5f} (Uni_rel: {loss1:.5f}, Bi_rel: {loss2:.5f}), LR: {current_lr:.6f}")
print(f"\nTraining finished. Final Loss: {total_loss:.5f}")

# --- Contribution Calculation Section ---
def get_model_terms(model):
    if isinstance(model, StrainEnergy_ExpPower_C):
        k1=1.0+tf.exp(model.raw_log_k1); k2=1.5+tf.exp(model.raw_log_k2); k3=1.0+tf.exp(model.raw_log_k3); k4=1.5+tf.exp(model.raw_log_k4)
        i1=1.0+tf.exp(model.raw_log_i1); i2=1.0+tf.exp(model.raw_log_i2); i3=1.0+tf.exp(model.raw_log_i3); i4=1.0+tf.exp(model.raw_log_i4)
        a1=tf.exp(model.raw_log_a1); a2=tf.exp(model.raw_log_a2); a3=tf.exp(model.raw_log_a3); a4=tf.exp(model.raw_log_a4)
        a3_prime=tf.exp(model.raw_log_a3_prime); a4_prime=tf.exp(model.raw_log_a4_prime)
        k9=1.0+tf.exp(model.raw_log_k9); k10=1.5+tf.exp(model.raw_log_k10); k11=1.0+tf.exp(model.raw_log_k11); k12=1.5+tf.exp(model.raw_log_k12)
        b1=tf.exp(model.raw_log_b1); b2=tf.exp(model.raw_log_b2); b3=tf.exp(model.raw_log_b3); b4=tf.exp(model.raw_log_b4)
        b3_prime=tf.exp(model.raw_log_b3_prime); b4_prime=tf.exp(model.raw_log_b4_prime)
        return {
            "Pow(I1)": lambda I1,I2,I4,I6: model._term_power_law(I1,k1,i1,a1,model.three),
            "Pow(I2)": lambda I1,I2,I4,I6: model._term_power_law(I2,k2,i2,a2,model.three),
            "Exp(I1)": lambda I1,I2,I4,I6: model._term_exponential(I1,k3,i3,a3_prime,a3,model.three),
            "Exp(I2)": lambda I1,I2,I4,I6: model._term_exponential(I2,k4,i4,a4_prime,a4,model.three),
            "Pow(I4)": lambda I1,I2,I4,I6: model._term_identity_scaled(I4,k9,b1,model.one),
            "Pow(I6)": lambda I1,I2,I4,I6: model._term_identity_scaled(I6,k10,b2,model.one),
            "Exp(I4)": lambda I1,I2,I4,I6: model._term_exponential_no_i(I4,k11,b3_prime,b3,model.one),
            "Exp(I6)": lambda I1,I2,I4,I6: model._term_exponential_no_i(I6,k12,b4_prime,b4,model.one),
        }
    elif isinstance(model, StrainEnergy_SinhCoshPower_C):
        k1=1.0+tf.exp(model.raw_log_k1);k2=1.5+tf.exp(model.raw_log_k2);k5=1.0+tf.exp(model.raw_log_k5);k6=1.5+tf.exp(model.raw_log_k6);k7=1.0+tf.exp(model.raw_log_k7);k8=1.5+tf.exp(model.raw_log_k8);k13=1.0+tf.exp(model.raw_log_k13);k14=1.5+tf.exp(model.raw_log_k14);k15=1.0+tf.exp(model.raw_log_k15);k16=1.5+tf.exp(model.raw_log_k16)
        i1=1.0+tf.exp(model.raw_log_i1);i2=1.0+tf.exp(model.raw_log_i2);i5=1.0+tf.exp(model.raw_log_i5);i6=1.0+tf.exp(model.raw_log_i6);i7=1.0+tf.exp(model.raw_log_i7);i8=1.0+tf.exp(model.raw_log_i8)
        a1=tf.exp(model.raw_log_a1);a2=tf.exp(model.raw_log_a2);a5=tf.exp(model.raw_log_a5);a6=tf.exp(model.raw_log_a6);a7=tf.exp(model.raw_log_a7);a8=tf.exp(model.raw_log_a8)
        a5_prime=tf.exp(model.raw_log_a5_prime);a6_prime=tf.exp(model.raw_log_a6_prime);a7_prime=tf.exp(model.raw_log_a7_prime);a8_prime=tf.exp(model.raw_log_a8_prime)
        b5=tf.exp(model.raw_log_b5);b6=tf.exp(model.raw_log_b6);b7=tf.exp(model.raw_log_b7);b8=tf.exp(model.raw_log_b8)
        b5_prime=tf.exp(model.raw_log_b5_prime);b6_prime=tf.exp(model.raw_log_b6_prime);b7_prime=tf.exp(model.raw_log_b7_prime);b8_prime=tf.exp(model.raw_log_b8_prime)
        return {
            "Pow(I1)":  lambda I1,I2,I4,I6: model._term_power_law(I1,k1,i1,a1,model.three),
            "Pow(I2)":  lambda I1,I2,I4,I6: model._term_power_law(I2,k2,i2,a2,model.three),
            "Cosh(I1)": lambda I1,I2,I4,I6: model._term_cosh_minus_one_with_i(I1,k5,i5,a5_prime,a5,model.three),
            "Cosh(I2)": lambda I1,I2,I4,I6: model._term_cosh_minus_one_with_i(I2,k6,i6,a6_prime,a6,model.three),
            "Sinh(I1)": lambda I1,I2,I4,I6: model._term_sinh_with_i(I1,k7,i7,a7_prime,a7,model.three),
            "Sinh(I2)": lambda I1,I2,I4,I6: model._term_sinh_with_i(I2,k8,i8,a8_prime,a8,model.three),
            "Cosh(I4)": lambda I1,I2,I4,I6: model._term_cosh_minus_one(I4,k13,b5_prime,b5,model.one),
            "Cosh(I6)": lambda I1,I2,I4,I6: model._term_cosh_minus_one(I6,k14,b6_prime,b6,model.one),
            "Sinh(I4)": lambda I1,I2,I4,I6: model._term_sinh(I4,k15,b7_prime,b7,model.one),
            "Sinh(I6)": lambda I1,I2,I4,I6: model._term_sinh(I6,k16,b8_prime,b8,model.one),
        }

def calculate_stress_contributions(lambda_values, model, task_type):
    terms = get_model_terms(model)
    contributions = {}
    print(f"\nCalculating contributions for {task_type}...")
    for name, W_func in terms.items():
        if task_type == 'uniaxial':
            stress_contribution = predict_uniaxial_p11(lambda_values, model, W_func)
        else: # biaxial
            stress_contribution = predict_biaxial_p22(lambda_values, model, W_func)
        contributions[name] = stress_contribution.numpy()
        print(f"  - Calculated contribution for term: {name}")
    return contributions

# --- Run Calculation and Save ---
lambda1_plot_tf = tf.constant(np.linspace(1.0, 2.5, 200), dtype=tf.float64)
lambda2_plot_tf = tf.constant(np.linspace(1.0, 2.5, 200), dtype=tf.float64)
p11_contributions = calculate_stress_contributions(lambda1_plot_tf, model, 'uniaxial')
p22_contributions = calculate_stress_contributions(lambda2_plot_tf, model, 'biaxial')

df_contrib = pd.DataFrame()
df_contrib['lambda1'] = lambda1_plot_tf.numpy()
for name, values in p11_contributions.items():
    df_contrib[f'P11_{name}'] = values
df_contrib['lambda2'] = lambda2_plot_tf.numpy()
for name, values in p22_contributions.items():
    df_contrib[f'P22_{name}'] = values

output_filename = f'calender_contributions_{MODEL_TYPE}.csv'
df_contrib.to_csv(output_filename, index=False)
print(f"\nSuccessfully saved stress contributions to '{output_filename}'")

Using device: /CPU:0 for model type: exp_power
--- Training exp_power model for Calender Tissue ---
Epoch     0, Loss: 305114545.61564 (Uni_rel: 22008962.47473, Bi_rel: 23512810.40599), LR: 0.003000
Epoch  1000, Loss: 0.25703 (Uni_rel: 0.07580, Bi_rel: 0.01213), LR: 0.003000
Epoch  2000, Loss: 0.06321 (Uni_rel: 0.02831, Bi_rel: 0.00237), LR: 0.002700
Epoch  3000, Loss: 0.04767 (Uni_rel: 0.02095, Bi_rel: 0.00164), LR: 0.002700
Epoch  4000, Loss: 0.03932 (Uni_rel: 0.01725, Bi_rel: 0.00134), LR: 0.002430
Epoch  5000, Loss: 0.02551 (Uni_rel: 0.01403, Bi_rel: 0.00069), LR: 0.002430
Epoch  6000, Loss: 0.01328 (Uni_rel: 0.00602, Bi_rel: 0.00053), LR: 0.002187
Epoch  7000, Loss: 0.01175 (Uni_rel: 0.00131, Bi_rel: 0.00083), LR: 0.002187
Epoch  8000, Loss: 0.00848 (Uni_rel: 0.00136, Bi_rel: 0.00057), LR: 0.001968
Epoch  9000, Loss: 0.00768 (Uni_rel: 0.00121, Bi_rel: 0.00051), LR: 0.001968
Epoch 10000, Loss: 0.00830 (Uni_rel: 0.00095, Bi_rel: 0.00059), LR: 0.001771
Epoch 11000, Loss: 0.01006 (Uni

  - Calculated contribution for term: Pow(I4)


  - Calculated contribution for term: Pow(I6)
  - Calculated contribution for term: Exp(I4)
  - Calculated contribution for term: Exp(I6)

Calculating contributions for biaxial...
  - Calculated contribution for term: Pow(I1)
  - Calculated contribution for term: Pow(I2)
  - Calculated contribution for term: Exp(I1)
  - Calculated contribution for term: Exp(I2)


  - Calculated contribution for term: Pow(I4)


  - Calculated contribution for term: Pow(I6)
  - Calculated contribution for term: Exp(I4)
  - Calculated contribution for term: Exp(I6)

Successfully saved stress contributions to 'calender_contributions_exp_power.csv'


In [6]:
# ==============================================================================
# ===== NEW SECTION: EXTRACT AND PRINT FINAL LEARNED PARAMETERS ==============
# ==============================================================================

print("\n" + "="*60)
print("      Final Learned Model Parameters (Raw Log Form)")
print("="*60)
# This prints the raw values that the optimizer sees.
for v in model.trainable_variables: # Assuming your trained model is named 'model'
    print(f"{v.name:20s}: {v.numpy():.8f}")

print("\n" + "="*60)
print("   Transformed Model Parameters (Physical Interpretable Values)")
print("="*60)

# --- Extract and Transform Parameters specific to the Exp-Power model ---
k1 = 1.0 + tf.exp(model.raw_log_k1).numpy()
k2 = 1.5 + tf.exp(model.raw_log_k2).numpy()
k3 = 1.0 + tf.exp(model.raw_log_k3).numpy()
k4 = 1.5 + tf.exp(model.raw_log_k4).numpy()
i1 = 1.0 + tf.exp(model.raw_log_i1).numpy()
i2 = 1.0 + tf.exp(model.raw_log_i2).numpy()
i3 = 1.0 + tf.exp(model.raw_log_i3).numpy()
i4 = 1.0 + tf.exp(model.raw_log_i4).numpy()
a1 = tf.exp(model.raw_log_a1).numpy()
a2 = tf.exp(model.raw_log_a2).numpy()
a3 = tf.exp(model.raw_log_a3).numpy()
a4 = tf.exp(model.raw_log_a4).numpy()
a3_prime = tf.exp(model.raw_log_a3_prime).numpy()
a4_prime = tf.exp(model.raw_log_a4_prime).numpy()

k9 = 1.0 + tf.exp(model.raw_log_k9).numpy()
k10 = 1.5 + tf.exp(model.raw_log_k10).numpy()
k11 = 1.0 + tf.exp(model.raw_log_k11).numpy()
k12 = 1.5 + tf.exp(model.raw_log_k12).numpy()
b1 = tf.exp(model.raw_log_b1).numpy()
b2 = tf.exp(model.raw_log_b2).numpy()
b3 = tf.exp(model.raw_log_b3).numpy()
b4 = tf.exp(model.raw_log_b4).numpy()
b3_prime = tf.exp(model.raw_log_b3_prime).numpy()
b4_prime = tf.exp(model.raw_log_b4_prime).numpy()


# --- Print in a clean, organized table format ---
print(f"{'Parameter':<12} | {'Value':<15} | {'Parameter':<12} | {'Value'}")
print("-" * 55)
print(f"{'k1':<12} | {k1:<15.5f} | {'i1':<12} | {i1:<15.5f}")
print(f"{'k2':<12} | {k2:<15.5f} | {'i2':<12} | {i2:<15.5f}")
print(f"{'k3':<12} | {k3:<15.5f} | {'i3':<12} | {i3:<15.5f}")
print(f"{'k4':<12} | {k4:<15.5f} | {'i4':<12} | {i4:<15.5f}")
print(f"{'k9':<12} | {k9:<15.5f} |")
print(f"{'k10':<12} | {k10:<15.5f} |")
print(f"{'k11':<12} | {k11:<15.5f} |")
print(f"{'k12':<12} | {k12:<15.5f} |")
print("-" * 55)
print(f"{'a1':<12} | {a1:<15.5f} | {'b1':<12} | {b1:<15.5f}")
print(f"{'a2':<12} | {a2:<15.5f} | {'b2':<12} | {b2:<15.5f}")
print(f"{'a3':<12} | {a3:<15.5f} | {'b3':<12} | {b3:<15.5f}")
print(f"{'a4':<12} | {a4:<15.5f} | {'b4':<12} | {b4:<15.5f}")
print(f"{'a3_prime':<12} | {a3_prime:<15.5f} | {'b3_prime':<12} | {b3_prime:<15.5f}")
print(f"{'a4_prime':<12} | {a4_prime:<15.5f} | {'b4_prime':<12} | {b4_prime:<15.5f}")
print("="*60)


      Final Learned Model Parameters (Raw Log Form)
raw_log_k1          : -0.76134237
raw_log_k2          : -6.61749669
raw_log_i1          : -1.91737236
raw_log_i2          : -9.68820560
raw_log_a1          : -3.59958873
raw_log_a2          : -1.68899257
raw_log_k3          : -1.32550620
raw_log_k4          : -3.22068638
raw_log_i3          : -0.79624336
raw_log_i4          : -4.19095797
raw_log_a3          : -5.97637289
raw_log_a4          : -5.34329170
raw_log_a3_prime    : -3.64781758
raw_log_a4_prime    : -5.29524916
raw_log_k9          : 0.15210526
raw_log_k10         : -0.14570058
raw_log_b1          : -3.68273651
raw_log_b2          : -4.36455860
raw_log_k11         : -1.53415257
raw_log_k12         : 1.15807349
raw_log_b3          : -2.01139069
raw_log_b4          : -2.65077483
raw_log_b3_prime    : -2.60560234
raw_log_b4_prime    : -0.71744247

   Transformed Model Parameters (Physical Interpretable Values)
Parameter    | Value           | Parameter    | Value
--------------